Основные цели этого задания:

Научиться работать с документацией scikit-learn

Научиться готовить данные к обучению модели и к ее валидации

Научиться оценивать качество модели и выбирать лучшую модель, опираясь на метрики

Задача:

Построить модель машинного обучения для решения задачи регрессии: необходимо спрогнозировать прогресс заболевания диабетом через год после исходного уровня.

План решения:

В документации библиотеки scikit learn найдите, как загрузить датасет для построения модели прогноза прогрессирования заболевания через год после исходного уровня.

Выведите содержимое поля DESCR, которое вернет функция, загружающая датасет, чтобы изучить содержимое датасета.

Подготовьте данные к обучению моделей: отделите целевой признак от датасета, разделите датасет на тренировочную и валидационную части.

Обучите решающее дерево и оцените адекватность обученной модели. Для этого достаточно сравнить метрики качества с метриками, получаемыми от простой модели. Например, если бы мы подавали на вход среднее значение целевой переменной на обучающей части выборки.

Обучите линейную регрессию и оцените ее адекватность.

Выберите лучшую модель и обоснуйте свой выбор.

In [27]:
from sklearn.datasets import load_diabetes
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error
from sklearn.linear_model import LinearRegression

from matplotlib import pyplot as plt
import pandas as pd

In [40]:
pd.set_option('display.float_format', lambda x: f'{x:.2f}')

In [41]:
print(load_diabetes().DESCR)

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

:Number of Instances: 442

:Number of Attributes: First 10 columns are numeric predictive values

:Target: Column 11 is a quantitative measure of disease progression one year after baseline

:Attribute Information:
    - age     age in years
    - sex
    - bmi     body mass index
    - bp      average blood pressure
    - s1      tc, total serum cholesterol
    - s2      ldl, low-density lipoproteins
    - s3      hdl, high-density lipoproteins
    - s4      tch, total cholesterol / HDL
    - s5      ltg, possibly log of serum triglycerides level
    - s6      glu, blood sugar level

Note: Each of these 10 feature variables have bee

In [42]:
df = load_diabetes(as_frame=True).frame

In [43]:
df.sample(5)

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
171,-0.02,-0.04,-0.05,-0.10,-0.08,-0.06,-0.02,-0.04,-0.05,-0.05,74.00
94,-0.08,-0.04,-0.02,-0.01,-0.00,-0.01,0.07,-0.04,-0.04,-0.09,90.00
408,0.06,-0.04,-0.05,0.11,0.03,0.02,-0.02,0.02,0.06,0.04,189.00
67,0.04,0.05,-0.01,-0.02,-0.01,0.01,-0.01,-0.00,-0.01,0.00,97.00
179,-0.02,-0.04,-0.02,-0.01,0.02,0.04,-0.04,0.03,0.01,-0.01,151.00


Данные нормализованы

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   age     442 non-null    float64
 1   sex     442 non-null    float64
 2   bmi     442 non-null    float64
 3   bp      442 non-null    float64
 4   s1      442 non-null    float64
 5   s2      442 non-null    float64
 6   s3      442 non-null    float64
 7   s4      442 non-null    float64
 8   s5      442 non-null    float64
 9   s6      442 non-null    float64
 10  target  442 non-null    float64
dtypes: float64(11)
memory usage: 38.1 KB


Пропусков явных нет

In [45]:
df.duplicated().sum()

np.int64(0)

Дубликатов нет

In [46]:
df_features = df.drop(['target'], axis=1)
df_target = df['target']

display(df_features.shape)
display(df_target.shape)

(442, 10)

(442,)

In [47]:
features_train, features_test, target_train, target_test = train_test_split(
    df_features, df_target, test_size=.3, random_state=42
)

In [48]:
model = tree.DecisionTreeRegressor()

In [49]:
model.fit(features_train, target_train)

DecisionTreeRegressor()

In [50]:
predict_val = model.predict(features_test)

In [51]:
root_mean_squared_error(target_test, predict_val)

np.float64(74.4530431822484)

In [52]:
mean_target = target_train.mean()
mean_predictions = [mean_target] * len(target_test)
mean_predictions[:5]

[np.float64(153.90291262135923),
 np.float64(153.90291262135923),
 np.float64(153.90291262135923),
 np.float64(153.90291262135923),
 np.float64(153.90291262135923)]

In [53]:
root_mean_squared_error(target_test, mean_predictions)

np.float64(73.70811591190015)

In [54]:
model2 = tree.DecisionTreeRegressor(max_depth=5)
model2.fit(features_train, target_train)
predict = model2.predict(features_test)
root_mean_squared_error(target_test, predict)

np.float64(64.04922930746785)

**Небольшой вывод**
- Деревья решений могут легко переобучаться
- Если не накладывать ограничение на глубину дерева, то даже среднее значение в качестве предсказанного будет давать меньшую ошибку
- Когда мы сделали ограничение на глубину дерева, ошибка уменьшилась

In [55]:
model_linear_regression = LinearRegression()
model_linear_regression.fit(features_train, target_train)

LinearRegression()

In [56]:
predict = model_linear_regression.predict(features_test)
root_mean_squared_error(target_test, predict)

np.float64(53.120156070942706)

**Вывод**
- Линейная регрессия показала более точные результаты
- Деревьям решений важно задавать ограничение на глубину, иначе модель будет переобучаться и давать менее точные ответы
- Использовать среднее значение в качестве предсказаний нельзя, предсказания будут неточными